## Objetivo del proyecto: 
El propósito de este proyecto es poder elaborar un analisis exploratorio de
los libros empleando diversos algoritmos que estudiamos durante el curso. Analisis estadísticos,
algoritmos no supervisados y elementos de visualización serán sus principales herramientas para
este analisis. Adicionalmente cualquier algoritmo o herramienta que no hayamos visto durante el
curso es válida, sin embargo, es importante que conozcan el funcionamiento y puedan explicar lo
que se está realizando con claridad.

## Resultados del proyecto:
Los elementos mínimos que su analisis debe cumplir son los siguientes:

1. Contestar los siguientes cuestionamientos<br>
    a. ¿Cuántos y cuales son los idiomas que identificaron en los libros?<br>
    b. ¿Cuántos libros tenemos por cada idioma?<br>
    c. ¿Cuántos géneros tenemos?<br>
        i. ¿Qué proceso utilizaron para identificar el (los) géneros del libro<br>
    d. ¿Encontraron algún otro patrón o elemento que consideren importante mencionar
en su analisis?<br>

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [40]:
# Para poder analizar los datos preprocesamos los textos a un .csv con los campos "campo: valor" antes de *** START OF THE PROJECT GUTENBERG EBOOK <<TITLE>> **
import glob
import regex as re

def create_csv(csv_name: str):
    # Dataframe para los libros
    df = pd.DataFrame(columns = ["Filename", "ID", "Title", "Author", "Language", "Release Date", "Subjects"])

    libros = glob.glob('./Gutenberg/*.txt')

    # Iteramos por cada .txt
    for i, filename in enumerate(libros):
        row = [filename]
        features = {"Title": None, "Author": None, "Language": None, "Release Date": None}

        with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
            # Itera sobre cada linea de texto
            for line in f:
                # Itera sobre las features excepto "Release Date"
                for feat in set(features) - {"Release Date"}:

                    match = re.search(fr"(?<={feat}:\s).*(?=$)", line)
                    if match:
                        features[feat] = match.group(0)

                # Matchea Release Date
                match = re.search(r"(?<=Release Date:\s).*(?=\s\[.*\]$)", line)
                if match:
                    features["Release Date"] = match.group(0)

                # Si ya tenemos todos los features, dejar de iterar por lineas de texto
                if None not in features.values():
                    break
                
                
        # Conseguir generos
        id = re.search(r'((?<!-)((?<=pg)|(?<=^)))\d*((?=.txt)|(?=-0.txt))' ,filename).group(0)
        
        row.append(id)
        
        row.extend(features.values())
        
        row.append("TODO")
        if None not in row:
            df.loc[len(df)] = row

    df.set_index("Filename")
    df.to_csv(csv_name, index=False)
    return df

In [41]:
import os.path

fname = 'gutenberg.csv'

# importar de csv si ya existe
if os.path.isfile(fname):
    df = pd.read_csv(fname)
else:
    df = create_csv(fname)
    
df

,Filename,ID,Title,Author,Language,Release Date,Subjects
0,./Gutenberg\24747-0.txt,24747,Samlede værker,Jeppe Aakjær,Danish,"March 3, 2008",TODO
1,./Gutenberg\27566-0.txt,27566,La Force,Paul Adam,French,"December 19, 2008",TODO
2,./Gutenberg\33378-0.txt,33378,Au soleil de juillet (1829-1830),Paul Adam,French,"August 8, 2010",TODO
3,./Gutenberg\36460-0.txt,36460,Cœur-de-panthère,Gustave Aimard,French,"June 18, 2011",TODO
4,./Gutenberg\39331-0.txt,39331,Histoire des salons de Paris (Tome 1/6),"Laure Junot, duchesse d'Abrantès",French,"April 1, 2012",TODO
...,...,...,...,...,...,...,...
193,./Gutenberg\pg70639.txt,70639,"The Life and Adventures of Alexander Selkirk, ...",John Howell,English,"April 24, 2023",TODO
194,./Gutenberg\pg70640.txt,70640,Third Base Thatcher,Everett (Deacon) Scott,English,"April 24, 2023",TODO
195,./Gutenberg\pg70642.txt,70642,"Historical Vignettes, 1st Series",Bernard Capes,English,"April 25, 2023",TODO
196,./Gutenberg\pg70643.txt,70643,Coca and Cocaine,William Martindale,English,"April 26, 2023",TODO


In [33]:
# Separamos libros por idioma
languages = df['Language'].unique()

# Diccionario con los dataframes por idioma
df_langs = {}
for lang in languages:
    df_langs[lang] = df[df['Language'] == lang]


In [12]:
# Hacemos análisis de texto por idioma con textstat
# Basado en parte en https://www.kaggle.com/code/michaelrussell4/gutenberg-book-genre-feature-engineering



2. Elemento de visualización<br>
    a. ¿Cuáles son las palabras con mayor frecuencia en cada idioma?<br>
    b. ¿Es posible agrupar los libros mediante cluster de tal forma que se ilustren los
    grupos por idioma?<br>
        i. De ser así genera una gráfica que ilustre esto<br>
        ii. Caso contario explica el motivo<br>
    c. Genera una grafica que ilustre la identificación de géneros en los libros